# Chapter 3

The aim of this chapter is dive deeper into the different considerations of using objects and virtual functions as pay-offs. 

### The parent class

In [1]:
class PayOff():
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

### The new inhereted classes

In [10]:
class PayOffCall(PayOff):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)

In [5]:
class PayOffPut(PayOff):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(self.__Strike - Spot,0)

### Putting everything together

In [6]:
from random import random
from math import exp, sqrt, log

In [7]:
def SimpleMonteCarlo2(thePayOff,
                     Expiry,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = GetOneGaussianByBoxMuller()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = thePayOff(thisSpot)
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean      

In [8]:
def GetOneGaussianBySummation():
    result = 0
    for j in range(12):
        result += random()
    result -= 6
    return result

def GetOneGaussianByBoxMuller():
    sizeSquared = 1
    while sizeSquared >= 1:
        x = 2*random() - 1
        y = 2*random() - 1
        sizeSquared = x*x + y*y
    
    result = x*sqrt(-2*log(sizeSquared)/sizeSquared)
    return result

In [24]:
callPayOff = PayOffCall(40)
callPrice = SimpleMonteCarlo2(callPayOff,0.5,42,0.2,0.1,10000)
print(f'Call price: {round(callPrice,2)}, for 10000 paths')

Call price: 4.82, for 10000 paths


### Adding extra pay-offs without changing existing code

Now we will add another option without changing anything, in this case a double digital.

In [35]:
class PayOffDoubleDigital(PayOff):
    def __init__(self, LowerLevel_, UpperLevel_):
        self.__LowerLevel = LowerLevel_
        self.__UpperLevel = UpperLevel_
        
    def __call__(self,Spot):
        if Spot <= self.__LowerLevel or Spot >=  self.__UpperLevel:
            return 0
        else:
            return 1                  

We will run the code again now with this class.

In [36]:
DoubleDigitalPayOff = PayOffDoubleDigital(100,120)
DoubleDigitalPrice = SimpleMonteCarlo2(DoubleDigitalPayOff,1,100,0.2,0.05,1000000)
print(f'Double digital option: {round(DoubleDigitalPrice,4)}, for 1000000 paths')

Double digital option: 0.3206, for 1000000 paths


### Exercises

**Exercise 3.1** Write an inhereted class that does power options, and use it to price some.

In [38]:
class PayOffPowerCall(PayOff):
    def __init__(self, PowerExponent_, Strike_):
        self.__PowerExponent = PowerExponent_
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(pow(Spot,self.__PowerExponent)-self.__Strike,0) 

In [39]:
class PayOffPowerPut(PayOff):
    def __init__(self, PowerExponent_, Strike_):
        self.__PowerExponent = PowerExponent_
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(self.__Strike-pow(Spot,self.__PowerExponent),0)  

In [58]:
PowerCallPayOff = PayOffPowerCall(1,40)
PowerCallPayOffPrice = SimpleMonteCarlo2(PowerCallPayOff,0.5,42,0.2,0.1,10000)
print(f'Power Call option: {round(PowerCallPayOffPrice,4)}, for 10000 paths')

Power Call option: 4.7676, for 10000 paths


In [59]:
PowerPutPayOff = PayOffPowerPut(1,40)
PowerPutPayOffPrice = SimpleMonteCarlo2(PowerPutPayOff,0.5,42,0.2,0.1,10000)
print(f'Double digital option: {round(PowerPutPayOffPrice,4)}, for 10000 paths')

Double digital option: 0.8058, for 1000 paths


**Exercise 3.2** Implement an interface in which the user inputs a stromg and this is turned into a pay-pff class.

To implement the interface we just have to use an if else statement inside a function that has as input a dictionary with all the arguments necessary.

In [60]:
def interface(arguments):
    OptionType = arguments['OptionType']
    
    if OptionType == 'call':
        return PayOffCall(arguments['Strike'])
    
    elif OptionType == 'put':
        return PayOffPut(arguments['Strike'])
    
    elif OptionType == 'DoubleDigital' :
        return PayOffDoubleDigital(arguments['LowerLevel'],arguments['UpperLevel'])
    
    elif OptionType == 'Power Call':
        return PayOffPowerCall(arguments['Exponent'],arguments['Spot'])
    
    elif OptionType == 'Power Put':
        return PayOffPowerPut(arguments['Exponent'],arguments['Spot'])       

In [65]:
PowerPutPayOff = interface({'OptionType':'Power Put', 'Exponent':1, 'Spot':40})
PowerPutPayOffPrice = SimpleMonteCarlo2(PowerPutPayOff,0.5,42,0.2,0.1,10000)
print(f'Double digital option: {round(PowerPutPayOffPrice,4)}, for 10000 paths')

Double digital option: 0.8048, for 10000 paths


**Exercise 3.3** In the evil boss's list of demands in Chapter 1, try to identify as many inheretance relationships as possible.

This process is quite straightforward so it will not be solved here.